#### Place cell identification pipeline


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sps
from scipy.ndimage import gaussian_filter
from sklearn.feature_selection import mutual_info_regression as mi_skl
import seaborn as sns
from scipy import stats
import time
from matplotlib import gridspec
import os 

color_dic = {'PRE':'midnightblue', 
             'SAM':'lightseagreen', 
             'CHO':'deepskyblue',
             'Trajectory':'lightgrey',
             'Startbox':'red',
             'SW1':'turquoise', 
             'SW2':'mediumpurple', 
             'SW3':'cornflowerblue'} 

In [4]:
#Load data
ANIMALS = ['H0465','H0466', 'H0493', 'H2202','H2203','H0487'] #Animal IDS
#-------------------------------------------------------------------------------------------
#Path to data directory (dataDir is the only one that needs adapting)
codeDir = os.getcwd()
len2del = sum([len(i) for i in codeDir.split('/')[-2:]])+1
homeDir = codeDir[:-len2del]
dataDir = codeDir[:-sum([len(i) for i in codeDir.split('/')[-3:]])-2] + 'animal_data/'
pcDIR = homeDir + "/Data/Place cell meta" #Where to save place cell table

TRIALS = pd.read_csv(homeDir + "/Data/Trial_breakdown.csv") #Trial meta info 
SaveTo2 = homeDir+'Figures\Figure 2 Calcium imaging' #Where to save figures

In [5]:
def arena_binned(x, y, xdim, ydim, pix_cm, pf_cm):
    """Creates occupancy maps by bibning arena counting bin occupancy
    INPUTS
    ------
    x = x pixel coordinates, 
    y = y pixel coordinates
    xdim = pixel coordinates for the arena sides e.g. [0, 500]
    ydim = pixel coordinates for the arena top and bottom e.g. [0, 500],
    pix_cm = conversion of number of pixels to cm
    pf_cm = size of the bins to use (cm)
    
    RETURNS:
    ------
    linS = Linearised bin occupancy
    occMap = 2D matrix of occupancy values
    nBnx = number of bins in x
    nBny = number of bins in y 
    bx = bin edges x
    by = bin edges y
    """
    
    d_x = xdim[0] - xdim[1] # xdim[0] = max, xdim[1] = min
    d_y = ydim[0] - ydim[1] # ydim[0] = max, ydim[1] = min
    nBnx = int((d_x/(pix_cm*pf_cm)))
    nBny = int((d_y/(pix_cm*pf_cm)))
    bn_x = [int(i) for i in np.linspace(xdim[0],xdim[1],nBnx)]
    bn_y = [int(i) for i in np.linspace(ydim[0], ydim[1],nBny)]

    bx = ((x-bn_x[0])/(bn_x[1]-bn_x[0])).astype(int)
    by = ((y-bn_y[0])/(bn_y[1]-bn_y[0])).astype(int)
    bx[bx>=20] = 19
    by[by>=20] = 19
    S = np.vstack((bx,by))
    linS = np.ravel_multi_index(S,(nBnx,nBny))
    occMap = sps.csr_matrix((np.ones(len(bx)),(bx,by)),shape=(nBnx,nBny),dtype=float).todense()
    return linS.astype(float), occMap, nBnx, nBny, bx, by


def bursting_check(linSpf, linS, e_trace, tr_delay):
    """Checks if the cell randomly bursts in a bin (which may equate to high mutual information)
    or if it is firing consistently across bin visits
    
    INPUTS
    ------
    linSpf = linearised firing in each bin
    linS = linearised occupancy in each bin
    e_trace = event traces
    tr_delay = the delay between bin entries to consider (e.g if the cell is bursting 
    and the animals leaves a bin for 1 second, then returns it may be mistinterpreted as a place cell  )
    
    RETURNS:
    tr = the number of times the animal traverses that bin
    rdm_burst = the number of traversals that the cell fires
    """
    #Place field timestamps
    pft_idxs = [i for i in range(len(linS)) if linS[i] in linSpf]
  
    #Seperate timestamps into traverals
    tr_idxs_all = []; tr_idxs = []
    i = 0
    while i < len(pft_idxs)-1:
        tr = pft_idxs[i+1]-pft_idxs[i]
        tr_idxs.append(pft_idxs[i])

        if tr > tr_delay: #acceptable delay between traversals
            tr_idxs_all.append(tr_idxs)
            tr_idxs = []
        i += 1
    
    if len(tr_idxs_all) > 0: #If 1 single traversal
        
        #Calculate the number of traversals with events
        tr = 0
        for i in tr_idxs_all:
            n_events = sum(e_trace[i])
            if n_events > 0:
                tr += 1
    else:
        tr = 1; tr_idxs_all = [[0]]
    
    return tr,(tr/len(tr_idxs_all))*100

In [13]:
#PLACE CELL INCLUSION CRITERIA
n_events = 3 # CONDITION 1: set n events for a cell to be considered
perc_thresh = 99 # CONDITION 2: Set place cell spatial info percentile 
n_tr = 3 # CONDITION 3: set number of traversals to be considered
tr_thresh = 20 # CONDITION 4: % traversals that cell fires <-- accounts for rdm bursting

#ADDITIONAL PARAMS
xdim = [575, 175] # Set arena x dimensions (northwest - (-20,-20), 320x320)
ydim = [450,50] # Set Arena y dimensions
pix_cm = 5 # Set how many pixels = 1 cm
pf_cm = 4 # Set bin size for place field (cms)
shuffles = 5000 # set number of shuffles to create SI distribution
tr_delay = 40 # acceptable delay between traversals (frames)

In [ ]:
for animal in ANIMALS:
    animDIR = os.path.join(dataDir, str(animal),'01_Aligned_LR')
    for file in os.listdir(animDIR):
        if file != 'traces' and file.endswith('.csv'):
            session, stage = file.split('_')[1:3]
            print(animal, session, stage)
            #Read in aligned events df
            meta = pd.read_csv(animDIR+'/'+file)

            #Remove all start box activity
            a_meta = meta[meta['position']!=12]
            a_meta = a_meta[a_meta['position']!=str(12)]

            #Only analyse moving points # INCLUDE SPEED FUNCTION HERE
            mov_meta = a_meta[a_meta['Movement status']=='moving']
            x = mov_meta['x'].values.astype(float)
            y = mov_meta['y'].values.astype(float)

            #Bin and linearize occupancy
            linS, occMap, nBnx, nBny, bx, by = arena_binned(x = x, y = y,
                                                            xdim = xdim, ydim = ydim,
                                                            pix_cm = pix_cm, pf_cm = pf_cm)

            #Isolate and binarize events
            e_traces = mov_meta[mov_meta[[col for col in mov_meta if col.startswith(' C')]]
                                .loc[:,~(mov_meta[[col for col in mov_meta if col.startswith(' C')]] == " nan").any()].columns.values]
            cell_IDS = e_traces.columns.values
            e_traces[e_traces > 0] = 1 # OPTIONAL: Binarized events
            e_traces = e_traces.values.T

            criteria = np.zeros((len(e_traces),4));
            pcs = []; Percs = []; SI =[]
            for i, ev in enumerate(e_traces):
                print(i+1,'/',len(cell_IDS))

                #Calculate place map
                rateMap = sps.csr_matrix((ev,(bx,by)),shape=(nBnx,nBny),dtype=float).todense()
                placeMap = np.divide(rateMap,occMap); placeMap[np.isnan(placeMap)] = 0

                #CONDITION 1: Exclude cells that have less than n events
                if  len([e for e in ev if e > 0]) > n_events:
                    criteria[i][0] = int(1)

                    #Calculate SI and compare to distribution
                    res_eve = list(np.array(ev.reshape(-1,1)).astype(float))
                    si = mi_skl(res_eve, linS, discrete_features=True,n_neighbors = 3)
                    SI.append(si)

                    #Create distribution of SI with shuffled events
                    si_shuf = []
                    for s in range(shuffles): 
                        np.random.shuffle(res_eve)
                        si_shuf.append(mi_skl(res_eve, t, discrete_features=True, n_neighbors=3))
                        if s > 0 and s % 1000 == 0:
                            print(s, 'shuffles completed')

                    #Calculate percentile
                    perc = stats.percentileofscore(si_shuf, si)
                    Percs.append(perc)

                    #Account for n traversalss random bursting cells
                    s_placeMap = gaussian_filter(placeMap, sigma = 1)
                    sn_placeMap = np.array(s_placeMap)/np.max(s_placeMap)
                    sn_placeMap[sn_placeMap < 0.2] = 0
                    pf_idxs = np.argwhere(sn_placeMap > 0)
                    S_pf = np.vstack((pf_idxs[:,0],pf_idxs[:,1]))
                    linSpf = np.ravel_multi_index(S_pf,(nBnx,nBny))
                    tr, rdm_burst = bursting_check(linSpf, linS, ev, tr_delay)

                    # CONDITION 2: cell SI > Threshold are considered a Place cell
                    if perc >= perc_thresh:
                        criteria[i][1] = int(1)

                    #CONDITION 3: Number of traversals required   
                    if tr > n_tr: 
                        criteria[i][2] = int(1)

                    #CONDITION 4: number of traversals with cell firing  
                    if rdm_burst >= tr_thresh: 
                        criteria[i][3] = int(1)

                    #If all place cell criteria, then place cell identified
                    if sum(criteria[i]) == 4:
                        print(cell_IDS[i], 'Is a place cell with %d traversals' %tr)
                        pcs.append(1)
                    else:
                        pcs.append(0)         
                else:
                    pcs.append(0)
                    SI.append(0)
                    Percs.append(0)
            
            #Read in animal place cell df
            pcANI = pd.read_csv(os.path.join(pcDIR, animal + '_place_cell_meta.csv'))
            
            #Add stage to DF
            PC_st = pd.DataFrame({'Cell IDS':cell_IDS,'Place cells':pcs,
                                  'Criteria':list(criteria),
                                  'Percentile':Percs,
                                  'Place cells':pcs,
                                  'SI':SI, 
                                  'Session':session, 
                                  'Stage':stage})
            
            pcANI = pcANI.append(PC_st)
            pcANI.to_csv(os.path.join(pcDIR, '%s_place_cell_meta.csv' %(animal)), index=False)
            print(sum(pcs), 'out of %d cells have been identified as place cells' %len(pcs))
            print((sum(pcs)/len(pcs))*100, ' Percent of cells are place cells')

H0465 A01 CHO
1 / 74
2 / 74
3 / 74
4 / 74
5 / 74
6 / 74
7 / 74
8 / 74
9 / 74
10 / 74
11 / 74
12 / 74


<ipython-input-16-06f06973c1af>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  e_traces[e_traces > 0] = 1 # OPTIONAL: Binarized events
C:\Users\fgobbo\Anaconda3\lib\site-packages\pandas\core\frame.py:3089: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
<ipython-input-16-06f06973c1af>:38: RuntimeWarning: invalid value encountered in true_divide
  placeMap = np.divide(rateMap,occMap); placeMap[np.isnan(placeMap)] = 0


1000 shuffles completed
2000 shuffles completed
3000 shuffles completed
4000 shuffles completed
13 / 74
14 / 74
15 / 74
16 / 74
17 / 74
18 / 74
19 / 74
20 / 74
21 / 74
22 / 74
23 / 74
24 / 74
25 / 74
26 / 74
27 / 74
28 / 74
29 / 74
30 / 74
31 / 74
32 / 74
33 / 74
34 / 74
35 / 74
36 / 74
37 / 74


<ipython-input-16-06f06973c1af>:38: RuntimeWarning: invalid value encountered in true_divide
  placeMap = np.divide(rateMap,occMap); placeMap[np.isnan(placeMap)] = 0


1000 shuffles completed
2000 shuffles completed
3000 shuffles completed
4000 shuffles completed
38 / 74
39 / 74
40 / 74
41 / 74
42 / 74
43 / 74
44 / 74
45 / 74
46 / 74
47 / 74
48 / 74
49 / 74
50 / 74
51 / 74
52 / 74
53 / 74
54 / 74
55 / 74
56 / 74
57 / 74
58 / 74
59 / 74
60 / 74
61 / 74


...

<ipython-input-16-06f06973c1af>:38: RuntimeWarning: invalid value encountered in true_divide
  placeMap = np.divide(rateMap,occMap); placeMap[np.isnan(placeMap)] = 0


1000 shuffles completed
2000 shuffles completed
3000 shuffles completed
4000 shuffles completed
28 / 104
29 / 104


<ipython-input-16-06f06973c1af>:38: RuntimeWarning: invalid value encountered in true_divide
  placeMap = np.divide(rateMap,occMap); placeMap[np.isnan(placeMap)] = 0


1000 shuffles completed
2000 shuffles completed
3000 shuffles completed
4000 shuffles completed
30 / 104
31 / 104
32 / 104
33 / 104


<ipython-input-16-06f06973c1af>:38: RuntimeWarning: invalid value encountered in true_divide
  placeMap = np.divide(rateMap,occMap); placeMap[np.isnan(placeMap)] = 0
